# 5.2 Essential Functionality

接下来介绍pandas中的一些主要功能，这里只介绍一些经常用到的。

In [71]:
import pandas as pd

## 1 Reindex(重新索引)

重新排列index，并返回一个新的object。如果原对象不存在index则用NaN填充。对于DataFrame，reindex能更改行索引或列索引。

In [72]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'e', 'a', 'c'])
obj

d    4.5
e    7.2
a   -5.3
c    3.6
dtype: float64

In [73]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    NaN
c    3.6
d    4.5
e    7.2
dtype: float64

注意reindex和重新赋index是完全不一样的：

In [74]:
obj.index = ['a', 'b', 'c', 'd']
obj

a    4.5
b    7.2
c   -5.3
d    3.6
dtype: float64

## 2 Dropping Entries from an Axis (按轴删除记录) 

drop删去你制定的axis的值，并返回一个新的object。对于DataFrame，index能按行或列的axis来删除。

In [75]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [76]:
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [77]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

drop也可以不返回一个新的object，而是直接更改series or dataframe in-place:

In [78]:
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

## 3 Indexing, Selection, and Filtering(索引，选择，过滤)

series indexing使用series的index：

In [79]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [80]:
print(obj['b'])
# 用label来slicing(切片)的时候，和python的切片不一样的在于，会包括尾节点：
print(obj['b':'c'])

1.0
b    1.0
c    2.0
dtype: float64


当index非数字索引时，也可以进行整数索引和切片，整数表示行号，跟numpy的indexing一致：

In [81]:
print(obj[1])
print(obj[-1])

1.0
3.0


注意，当index是数字的时候，使用类似于-1，-2这样的负整数索引会产生歧义（例如，不知道-1是表示最后一行还是表示-1这个index本身），实际上，这个时候程序会解析成index本身，如果不存在这个index会引发程序报错。因此，建议尽量少用整数索引，如有需要，使用iloc

In [82]:
obj2 = pd.Series(np.arange(4.))
print(obj2[-1])

KeyError: -1

根据布尔条件，产生布尔Series：

In [ ]:
obj < 2

布尔索引，相当于筛出该布尔条件产生的布尔Series为True的项：

In [ ]:
obj[obj < 2]

而对于DataFrame，indexing同样可以通过一个整数值或label索引，但是与Series不同的是两者不等价，通过整数值索引进行的是行索引，通过label索引进行的是列索引：

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

In [ ]:
data['two']

In [ ]:
data[:2]

dataframe的indexing有一些比较特别的方式。比如通过布尔数组：

In [ ]:
data[data['three'] > 5]

布尔dataframe：

In [ ]:
data < 5

布尔索引

In [ ]:
data[data < 5] = 0
data

## 4 Arithmetic and Data Alignment (算数和数据对齐)

pandas一个有用的feature就是，不同index的obejct之间的算数计算。如果两个object相加，但他们各自的index并不相同，最后结果得到的index是这两个index的合集：

In [ ]:
s1 = pd.Series([7.3, -2.5, 3.4, np.nan], index=['a', 'c', 'd', 'e'])

In [ ]:
s2 = pd.Series([2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])

In [ ]:
s1

In [ ]:
s2

In [ ]:
s1 + s2

这种数据对齐的方式（internal data alignment）引入了很多缺失值。这些缺失值会被用在之后的算数计算中，一旦有NaN参与计算，结果一定为NaN。

在DataFrame中，数据对齐同时发生在行和列上：

In [ ]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])

In [ ]:
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [ ]:
df1

In [ ]:
df2

相加的结果就是两个DataFrame，行和列的合集：

In [ ]:
df1 + df2

因为'c'和'e'列都不在两个DataFrame里，所有全是缺失值。对于行，即使有相同的，但列不一样的话也会是缺失值。

如果两个DataFrame相加，而且没有column和row，结果会全是null：

In [ ]:
df1 = pd.DataFrame({'A': [1, 2]})
df2 = pd.DataFrame({'B': [3, 4]})

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1 - df2

### Operations between DataFrame and Series (DataFrame和Series之间的操作)

先举个numpy的例子帮助理解，可以考虑成一个二维数组和它的一行：

In [ ]:
arr = np.arange(12.).reshape((3, 4))
arr

In [ ]:
arr[0]

In [ ]:
arr - arr[0]

可以看到，这个减法是用在了每一行上。这种操作叫broadcasting，在Appendix A有更详细的解释。DataFrame和Series的操作也类似：

In [ ]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                    index=['Utah', 'Ohio', 'Texas', 'Oregon'])
series = frame.iloc[0]

In [ ]:
frame

In [ ]:
series

可以理解为series的index与dataframe的列匹配，broadcasting down the rows(向下按行广播):

In [ ]:
frame - series

如果一个index既不在DataFrame的column中，也不再series里的index中，那么结果也是合集：

In [ ]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
frame + series2

如果想要广播列，去匹配行，必须要用到算数方法，带参数指定axis：

In [ ]:
series3 = frame['d']
frame

In [ ]:
series3

In [ ]:
frame.sub(series3, axis='index')

axis参数就是用来匹配轴的。在这个例子里是匹配dataframe的row index(`axis='index` or `axis=0`)，然后再广播。

## 6 Function Application and Mapping (函数应用和映射)

numpy的ufuncs(element-wise的数组方法，即「逐个元素地」计算的方法)也能用在pandas的object上,它会作用于每一个元素：

In [ ]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'), 
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

In [ ]:
np.abs(frame)

另一个常用的操作是把一个用在一维数组上的函数，应用在一行或一列上。要用到DataFrame中的apply函数：

In [ ]:
f = lambda x: x.max() - x.min()
frame.apply(f)

这里函数f，计算的是一个series中最大值和最小值的差，在frame中的每一列，这个函数被调用一次。作为结果的series，它的index就是frame的column。

如果你传入`axis='column'`用于apply，那么函数会被用在每一行：

In [ ]:
frame.apply(f, axis='columns')

apply也可以返回一个含有多个值的series：

In [ ]:
frame

In [ ]:
def f(x): 
    return pd.Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

像是sum, mean这样的数组统计方法，DataFrame中已经集成了，所以没必要用apply。

## 8 Axis Indexes with Duplicate Labels (有重复label的轴索引)

我们看到的所有例子都有unique axis labels(index values),唯一的轴标签（索引值）。一些pandas函数（reindex）,需要label是唯一的，但这并是不强制的。比如下面有一个重复的索引：

In [ ]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

index的is_unique特性能告诉我们label是否是唯一的：

In [ ]:
obj.index.is_unique

数据选择对于重复label则表现有点不同。如果一个label有多个值，那么就会返回一个series, 如果是label只对应一个值的话，会返回一个标量：

In [ ]:
obj['a']

In [ ]:
obj['c']

这个选择的逻辑也应用于DataFrame：

In [ ]:
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df

In [ ]:
df.loc['b']